In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Low MI scores
for pp in ["cat2","cat3","cat4","cat6"]:
    train.pop(pp)
    test.pop(pp)
train.head()

,id,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,E,C,N,0.201470,-0.014822,0.669699,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,D,F,A,O,0.743068,0.367411,1.021605,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,D,D,A,F,0.742708,0.310383,-0.012673,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,D,E,C,K,0.429551,0.620998,0.577942,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,D,E,A,N,1.058291,0.367492,-0.052389,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [6]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat5,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,E,C,N,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,D,F,A,O,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,D,D,A,F,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,D,E,C,K,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,D,E,A,N,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [7]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [8]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_28,ohe_29,ohe_30,ohe_31,ohe_32,ohe_33,ohe_34,ohe_35,ohe_36,ohe_37
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [10]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [11]:
pseudo = pd.read_csv("../input/30dml-pseudo2/xgb_submission_k.txt")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [12]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [13]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
               'objective': 'reg:squarederror',
               'learning_rate':trial.suggest_uniform('learning_rate', 4.58e-2, 5.58e-2),
               'n_estimators':N_ESTIMATORS,
               'subsample': trial.suggest_uniform('subsample', 0.963, 1),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.0585, 0.0985),
              'max_depth':trial.suggest_int('max_depth', 5, 7),
              'lambda':trial.suggest_uniform('lambda', 87.16, 92.16),
              'alpha':trial.suggest_uniform('alpha', 15.7, 18.7),
           'min_child_weight':trial.suggest_uniform('min_child_weight', 218, 245),
           'importance_type': 'total_gain',
              'booster': 'gbtree',
            'tree_method': 'gpu_hist'
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  xgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_row(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = xgb.XGBRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        
    )


    xgb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, xgb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [14]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-31 00:49:55,295] A new study created in memory with name: no-name-457bcd4d-c67f-48ee-9395-454f2c25b396


[0]	validation_0-rmse:7.39342
[1000]	validation_0-rmse:0.71285
[2000]	validation_0-rmse:0.71057
[3000]	validation_0-rmse:0.71005
[4000]	validation_0-rmse:0.70985
[5000]	validation_0-rmse:0.70978
[5371]	validation_0-rmse:0.70977
[0]	validation_0-rmse:7.39343
[1000]	validation_0-rmse:0.71305
[2000]	validation_0-rmse:0.71056
[3000]	validation_0-rmse:0.71008
[4000]	validation_0-rmse:0.70993
[5000]	validation_0-rmse:0.70985
[5021]	validation_0-rmse:0.70985


[I 2021-08-31 00:50:58,141] Trial 0 finished with value: 0.7097679114099649 and parameters: {'learning_rate': 0.04966577780453602, 'subsample': 0.9648127717874132, 'colsample_bytree': 0.07168710763104424, 'max_depth': 5, 'lambda': 91.22081435431974, 'alpha': 17.754656836593437, 'min_child_weight': 221.69403254108673}. Best is trial 0 with value: 0.7097679114099649.


[0]	validation_0-rmse:7.37282
[1000]	validation_0-rmse:0.71103
[2000]	validation_0-rmse:0.71001
[3000]	validation_0-rmse:0.70987
[3199]	validation_0-rmse:0.70988
[0]	validation_0-rmse:7.37282
[1000]	validation_0-rmse:0.71109
[2000]	validation_0-rmse:0.71005
[2849]	validation_0-rmse:0.70992


[I 2021-08-31 00:51:44,940] Trial 1 finished with value: 0.7098273441969993 and parameters: {'learning_rate': 0.052340477046363285, 'subsample': 0.9807212347305155, 'colsample_bytree': 0.08280607157550475, 'max_depth': 6, 'lambda': 87.16124382720635, 'alpha': 16.28372167773897, 'min_child_weight': 226.99794288010568}. Best is trial 0 with value: 0.7097679114099649.


...

[0]	validation_0-rmse:7.35463
[1000]	validation_0-rmse:0.71243
[2000]	validation_0-rmse:0.71046
[3000]	validation_0-rmse:0.70998
[4000]	validation_0-rmse:0.70984
[4958]	validation_0-rmse:0.70981
[0]	validation_0-rmse:7.35464
[1000]	validation_0-rmse:0.71259
[2000]	validation_0-rmse:0.71046
[3000]	validation_0-rmse:0.71003
[4000]	validation_0-rmse:0.70993
[4567]	validation_0-rmse:0.70990


[I 2021-08-31 01:14:54,372] Trial 28 finished with value: 0.7098034097421807 and parameters: {'learning_rate': 0.05470153512847735, 'subsample': 0.9683662287012604, 'colsample_bytree': 0.05857092074655468, 'max_depth': 5, 'lambda': 89.42610936460609, 'alpha': 17.916838640824544, 'min_child_weight': 227.1647204301212}. Best is trial 23 with value: 0.7097668779647643.


[0]	validation_0-rmse:7.36367
[1000]	validation_0-rmse:0.71176
[2000]	validation_0-rmse:0.71037
[3000]	validation_0-rmse:0.71001
[4000]	validation_0-rmse:0.70993
[4570]	validation_0-rmse:0.70992
[0]	validation_0-rmse:7.36366
[1000]	validation_0-rmse:0.71179
[2000]	validation_0-rmse:0.71035
[3000]	validation_0-rmse:0.70999
[4000]	validation_0-rmse:0.70990
[4538]	validation_0-rmse:0.70989


[I 2021-08-31 01:15:51,910] Trial 29 finished with value: 0.7098417256919911 and parameters: {'learning_rate': 0.05352921682961161, 'subsample': 0.9766654947328101, 'colsample_bytree': 0.07917109144913197, 'max_depth': 5, 'lambda': 90.84357414842718, 'alpha': 18.193390085605735, 'min_child_weight': 220.65099191384428}. Best is trial 23 with value: 0.7097668779647643.


Number of finished trials: 30
Best trial: {'learning_rate': 0.05510759280586885, 'subsample': 0.9684651346063825, 'colsample_bytree': 0.07434683610800047, 'max_depth': 5, 'lambda': 90.83558021534245, 'alpha': 18.68492800206771, 'min_child_weight': 223.4301110745184}


In [15]:
study.best_params

{'learning_rate': 0.05510759280586885,
 'subsample': 0.9684651346063825,
 'colsample_bytree': 0.07434683610800047,
 'max_depth': 5,
 'lambda': 90.83558021534245,
 'alpha': 18.68492800206771,
 'min_child_weight': 223.4301110745184}

# Log

====== Ordinal encoding =========

0.7104482581506356 no noise ver1

0.714284445715517 row-wise noise ver2

====== One-hot encoding =========

0.7103796124277987 no noise ver3

0.7100970391939901 no noise ver4 (narrow space)

0.709957588415761 no noise ver5 (narrow space) final

========================================== 
=========== Changed pseudo ===============

====== Ordinal encoding =========

0.7101944034015998 no noise ver9

====== One-hot encoding =========

0.7100303460873891 no noise ver8

0.7099357251489501 no noise ver10 (narrow space)

0.7099117436246878 no noise ver11 (narrow space)

0.7098921880994804 no noise ver12 (narrow space)